In [ ]:
import torch
import torchaudio
import pandas as pd
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import common_utils as utils
import jiwer

# 모델 및 프로세서 로드
model_path = "./final-wav2vec2-stt"
processor = Wav2Vec2Processor.from_pretrained(model_path)
model = Wav2Vec2ForCTC.from_pretrained(model_path)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# 테스트 데이터 로드
wav_dir = "/kaggle/input/foregin-english/원천데이터/TS_2. 영어/1. 한국일반"
label_dir = "/kaggle/input/foregin-english/라벨링데이터/2. 영어/1. 한국일반"

# 오디오 파일 목록
audio_paths = []
ground_truth = []
for file in os.listdir(wav_dir):
    if file.endswith(".wav"):
        file_stem = os.path.splitext(file)[0]
        wav_path = os.path.join(wav_dir, file)
        transcript = utils.get_transcript_from_json(file_stem, label_dir)
        if transcript:
            audio_paths.append(wav_path)
            ground_truth.append(transcript)

# 예측 수행
predictions = []
for audio_path in audio_paths:
    predictions.append(utils.transcribe(audio_path, model, processor))

# 결과 평가
results_df = pd.DataFrame({
    "audio_path": audio_paths,
    "ground_truth": ground_truth,
    "prediction": predictions
})

# WER(Word Error Rate) 계산
wer = jiwer.wer(ground_truth, predictions)
print(f"Word Error Rate (WER): {wer:.4f}")

# 샘플 결과 확인
print("\n샘플 예측 결과:")
for i in range(min(5, len(results_df))):
    print(f"\n음성 파일: {results_df.loc[i, 'audio_path']}")
    print(f"실제 텍스트: {results_df.loc[i, 'ground_truth']}")
    print(f"예측 텍스트: {results_df.loc[i, 'prediction']}")